In [1]:
%load_ext autoreload
%autoreload 2
import sys,os
import torch
current_directory = os.getcwd()
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(current_directory))))))
from datasets.weather_bench import WeatherDataset

In [2]:
device = ("cuda" if torch.cuda.is_available() else "cpu" )
device = torch.device(device)

weather = WeatherDataset(0, device=device, normalize=True)
# dataset.shape:  torch.Size([7309, 100, 1450])
original = weather.load()

데이터셋 불러오는 중...
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7305, latitude: 28, longitude: 28,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float32 39.0 38.75 38.5 ... 32.5 32.25
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float32 124.2 124.5 ... 130.8 131.0
  * time                      (time) datetime64[ns] 2016-12-31 ... 2021-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, latitude, longitude) float32 ...
    10m_v_component_of_wind   (time, latitude, longitude) float32 ...
    2m_temperature            (time, latitude, longitude) float32 ...
    geopotential              (time, level, latitude, longitude) float32 ...
    mean_sea_level_pressure   (time, latitude, longitude) float32 ...
    sea_surface_temperature   (time, latitude, longitude) float32 ...
    ...                        ...
    total_cloud

Processing futures: 14it [00:13,  1.04it/s]
Processing futures: 2it [00:01,  1.53it/s]


15.61906 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7305, longitude: 14, latitude: 33,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float64 21.0 22.5 24.0 ... 67.5 69.0
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float64 121.5 123.0 ... 139.5 141.0
  * time                      (time) datetime64[ns] 2016-12-31 ... 2021-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, longitude, latitude) float32 ...
    10m_v_component_of_wind   (time, longitude, latitude) float32 ...
    2m_temperature            (time, longitude, latitude) float32 ...
    geopotential              (time, level, longitude, latitude) float32 ...
    mean_sea_level_pressure   (time, longitude, latitude) float32 ...
    sea_surface_temperature   (time, longitude, latitude) float32 ...
    ...                        ...
    total_cloud_cov

Processing futures: 14it [00:10,  1.33it/s]
Processing futures: 2it [00:00,  6.24it/s]


11.11322 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7305, longitude: 17, latitude: 12,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float64 2.903 8.71 ... 60.97 66.77
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float64 90.0 95.62 ... 174.4 180.0
  * time                      (time) datetime64[ns] 2016-12-31 ... 2021-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, longitude, latitude) float32 ...
    10m_v_component_of_wind   (time, longitude, latitude) float32 ...
    2m_temperature            (time, longitude, latitude) float32 ...
    geopotential              (time, level, longitude, latitude) float32 ...
    mean_sea_level_pressure   (time, longitude, latitude) float32 ...
    sea_surface_temperature   (time, longitude, latitude) float32 ...
    ...                        ...
    total_cloud_cover 

Processing futures: 14it [00:09,  1.41it/s]
Processing futures: 2it [00:00, 14.58it/s]


10.23150 sec
======= RESULT SHAPE =======
result_dataset.shape:  torch.Size([7305, 100, 1450])


In [11]:
%load_ext autoreload
%autoreload 2
from models.VariableAnalyzer.datasets.dataset import CustomDataset
dataset = CustomDataset(original, 4 * 7, device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
print(dataset[3])

getdata:  0.00042 sec
getdata:  0.00297 sec
res:  0.00948 sec
tensor([[0.2695, 0.2715, 0.2703,  ..., 0.7818, 0.6788, 0.5188],
        [0.1810, 0.1773, 0.1726,  ..., 0.4673, 0.4184, 0.3465],
        [0.1628, 0.1569, 0.1506,  ..., 0.3565, 0.3355, 0.2872],
        ...,
        [0.3319, 0.3644, 0.3936,  ..., 0.2570, 0.4454, 0.6686],
        [0.2312, 0.4194, 0.4887,  ..., 0.2219, 0.3896, 0.6130],
        [0.3171, 0.0150, 0.0218,  ..., 0.1309, 0.2664, 0.4330]])


In [19]:
from torch.utils.data import DataLoader, Dataset

dataloader = DataLoader(dataset, 32, shuffle=True)

In [20]:
for batch in dataloader:
    print(batch.shape)

getdata:  0.00698 sec
getdata:  0.00315 sec
res:  0.00961 sec
getdata:  0.00019 sec
getdata:  0.00311 sec
res:  0.00410 sec
getdata:  0.00024 sec
getdata:  0.00329 sec
res:  0.00831 sec
getdata:  0.00030 sec
getdata:  0.00298 sec
res:  0.00867 sec
getdata:  0.00027 sec
getdata:  0.00421 sec
res:  0.01950 sec
getdata:  0.00037 sec
getdata:  0.00453 sec
res:  0.01503 sec
getdata:  0.00034 sec
getdata:  0.00302 sec
res:  0.01003 sec
getdata:  0.00024 sec
getdata:  0.00347 sec
res:  0.00509 sec
getdata:  0.00028 sec
getdata:  0.00296 sec
res:  0.00785 sec
getdata:  0.00037 sec
getdata:  0.00360 sec
res:  0.02018 sec
getdata:  0.00024 sec
getdata:  0.00299 sec
res:  0.00177 sec
getdata:  0.00019 sec
getdata:  0.00275 sec
res:  0.00305 sec
getdata:  0.00018 sec
getdata:  0.00290 sec


res:  0.02043 sec
getdata:  0.00023 sec
getdata:  0.00339 sec
res:  0.01272 sec
getdata:  0.00029 sec
getdata:  0.00321 sec
res:  0.00168 sec
getdata:  0.00019 sec
getdata:  0.00290 sec
res:  0.00197 sec
getdata:  0.00021 sec
getdata:  0.00275 sec
res:  0.00178 sec
getdata:  0.00019 sec
getdata:  0.00273 sec
res:  0.00173 sec
getdata:  0.00019 sec
getdata:  0.00328 sec
res:  0.00183 sec
getdata:  0.00019 sec
getdata:  0.00316 sec
res:  0.00172 sec
getdata:  0.00019 sec
getdata:  0.00274 sec
res:  0.00208 sec
getdata:  0.00023 sec
getdata:  0.00282 sec
res:  0.00186 sec
getdata:  0.00019 sec
getdata:  0.00325 sec
res:  0.00185 sec
getdata:  0.00018 sec
getdata:  0.00285 sec
res:  0.00166 sec
getdata:  0.00019 sec
getdata:  0.00290 sec
res:  0.00163 sec
getdata:  0.00019 sec
getdata:  0.00283 sec
res:  0.01786 sec
getdata:  0.00055 sec
getdata:  0.00322 sec
res:  0.02006 sec
getdata:  0.00039 sec
getdata:  0.00329 sec
res:  0.02043 sec
getdata:  0.00037 sec
getdata:  0.00367 sec
res:  0.

IndexError: list index out of range

In [15]:
from models.VariableAnalyzer.models.model import VariableAnalyzer

model = VariableAnalyzer(100, 4 * 7, 1450, 25, 3, 3)
print(model.get_tgt_mask(dataset[0]))

/workspace/Haea/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [26]:
def get_time_seq(src: torch.Tensor):
        time_seq = []
        for i in range(4 * 7):
            seq = [i for _ in range(100)]
            time_seq.extend(seq)
        return torch.tensor([time_seq for _ in range(src.size(0))])


print(get_time_seq(dataset[0:3, 1]))

(slice(0, 3, None), 1)


TypeError: list indices must be integers or slices, not tuple